# Permutation Flow-Shop Scheduling Problem

In [1]:
import json
import time

import pandas as pd

from bnbprob.pfssp import LazyBnB, PermFlowShop
from bnbprob.pfssp.cython.problem import PermFlowShop2MHalf
from bnbpy import configure_logfile

In [2]:
configure_logfile("pfssp-experiments-2mhalf-simple.log", mode="w")

In [3]:
import gc

gc.disable()

In [ ]:
solutions = {}
# problems = [f'ta{str(i).zfill(3)}' for i in range(41, 51, 1)]
problems = [f'ta{str(i).zfill(3)}' for i in range(1, 21, 1)] + [
    f'ta{str(i).zfill(3)}' for i in range(31, 51, 1) if i not in {42, 43, 47}
]

bnbs: dict[str, LazyBnB] = {}

for prob in problems:
    with open(
        f'./../data/flow-shop/{prob}.json', mode='r', encoding='utf8'
    ) as f:
        p = json.load(f)
        problem = PermFlowShop2MHalf.from_p(p, constructive='neh')
        bnb = LazyBnB(
            eval_node='in',
            rtol=0.0001,
            save_tree=False,
            queue_mode='dfs',  # 'dfs' or 'cycle'
        )
        bnb.log_row(prob)
        gc.collect()
        time.sleep(0.1)

        start = time.time()
        sol = bnb.solve(problem, maxiter=100_000_000, timelimit=3600 * 4)
        duration = time.time() - start
        solutions[prob] = (sol, duration)

        print(f'{prob}: {sol} in {duration:.2f}s ({bnb.explored} nodes)')

        bnbs[prob] = bnb
        time.sleep(0.1)

ta001: Status: OPTIMAL | Cost: 1278.0 | LB: 1278.0 in 0.01s (139 nodes)
ta002: Status: OPTIMAL | Cost: 1359.0 | LB: 1359.0 in 1.58s (26033 nodes)
ta003: Status: OPTIMAL | Cost: 1081.0 | LB: 1081.0 in 0.02s (348 nodes)
ta004: Status: OPTIMAL | Cost: 1293.0 | LB: 1293.0 in 0.01s (294 nodes)
ta005: Status: OPTIMAL | Cost: 1235.0 | LB: 1235.0 in 3.22s (89869 nodes)
ta006: Status: OPTIMAL | Cost: 1195.0 | LB: 1195.0 in 0.01s (372 nodes)
ta007: Status: OPTIMAL | Cost: 1234.0 | LB: 1234.0 in 0.07s (1238 nodes)
ta008: Status: OPTIMAL | Cost: 1206.0 | LB: 1206.0 in 0.03s (247 nodes)
ta009: Status: OPTIMAL | Cost: 1230.0 | LB: 1230.0 in 0.07s (1548 nodes)
ta010: Status: OPTIMAL | Cost: 1108.0 | LB: 1108.0 in 0.01s (55 nodes)
ta011: Status: OPTIMAL | Cost: 1582.0 | LB: 1582.0 in 12.98s (296723 nodes)
ta012: Status: OPTIMAL | Cost: 1659.0 | LB: 1659.0 in 42.20s (799554 nodes)
ta013: Status: OPTIMAL | Cost: 1496.0 | LB: 1496.0 in 28.94s (577586 nodes)
ta014: Status: OPTIMAL | Cost: 1377.0 | LB: 137

In [ ]:
# Export to dataframe
problems_ = []
ub = []
lb = []
gap = []
time_ = []
n_nodes = []

for key, val in solutions.items():
    sol, t = val
    problems_.append(key)
    ub.append(sol.cost)
    lb.append(sol.lb)
    gap.append(abs(sol.cost - sol.lb) / sol.cost)
    time_.append(t)
    n_nodes.append(bnbs[key].explored)


df = pd.DataFrame({
    "problem": problems_,
    "ub": ub,
    "lb": lb,
    "gap": gap,
    "nodes": n_nodes,
    "time": time_
})

df.to_excel("bench-pfssp-2mhalf-dfs-simple.xlsx")